In [1]:
from sklearn.metrics import  zero_one_loss,mean_absolute_error,r2_score, mean_squared_error
def get_mae(target, prediction):
    assert (target.shape == prediction.shape)

    return mean_absolute_error(target, prediction)

def get_mse(target, prediction):
    assert (target.shape == prediction.shape)

    return mean_squared_error(target, prediction)

def get_sae(target, prediction):
    assert (target.shape == prediction.shape)

    r = target.sum()
    r0 = prediction.sum()
    sae = abs(r0 - r) / r
    return sae

def get_nde(target, prediction):
    assert (target.shape == prediction.shape)

    error, squarey = [], []
    for i in range(len(prediction)):
        value = prediction[i] - target[i]
        error.append(value * value)
        squarey.append(target[i] * target[i])
    nde = sqrt(sum(error) / sum(squarey))
    return nde

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import logging
import numpy as np


def get_logger(filename):
    # Logging configuration: set the basic configuration of the logging system
    log_formatter = logging.Formatter(fmt='%(asctime)s [%(processName)s, %(process)s] [%(levelname)-5.5s]  %(message)s',
                                      datefmt='%m-%d %H:%M')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    # File logger
    file_handler = logging.FileHandler("{}.log".format(filename))
    file_handler.setFormatter(log_formatter)
    file_handler.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)
    # Stderr logger
    std_handler = logging.StreamHandler(sys.stdout)
    std_handler.setFormatter(log_formatter)
    std_handler.setLevel(logging.DEBUG)
    logger.addHandler(std_handler)
    return logger


def data_loader(inputs, targets, batch_size, shuffle=True):
    assert inputs.shape[0] == targets.shape[0]
    inputs_size = inputs.shape[0]
    if shuffle:
        random_order = np.arange(inputs_size)
        np.random.shuffle(random_order)
        inputs, targets = inputs[random_order, :], targets[random_order]
    num_blocks = int(inputs_size / batch_size)
    for i in range(num_blocks):
        yield inputs[i * batch_size: (i+1) * batch_size, :], targets[i * batch_size: (i+1) * batch_size]
    if num_blocks * batch_size != inputs_size:
        yield inputs[num_blocks * batch_size:, :], targets[num_blocks * batch_size:]


def multi_data_loader(inputs, targets, batch_size, shuffle=True):
    """
    Both inputs and targets are list of numpy arrays, containing instances and labels from multiple sources.
    """
    assert len(inputs) == len(targets)
    input_sizes = [data.shape[0] for data in inputs]
    max_input_size = max(input_sizes)
    num_domains = len(inputs)
    if shuffle:
        for i in range(num_domains):
            r_order = np.arange(input_sizes[i])
            np.random.shuffle(r_order)
            inputs[i], targets[i] = inputs[i][r_order, :], targets[i][r_order]
    num_blocks = int(max_input_size / batch_size)
    for j in range(num_blocks):
        xs, ys = [], []
        for i in range(num_domains):
            ridx = np.random.choice(input_sizes[i], batch_size)
            xs.append(inputs[i][ridx, :])
            ys.append(targets[i][ridx])
        yield xs, ys


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
def dataProvider(file,windowsize,stepsize,threshold):  
    dataframe=pd.read_csv(file,header=0)    
    np_array=np.array(dataframe)
    inputs, targets=np_array[:, 0], np_array[:, 1]
    offset = int(0.5 * (windowsize - 1.0))
#     window_num=inputs.size - 2 * offset
    features=list()
    labels=list()
    i=0
    while (i<=inputs.size-windowsize):
        data_in=inputs[i:i+windowsize]
        tar=targets[i+offset]
        features.append(data_in)
        labels.append(tar)
        i=i+stepsize
    X=np.array(features)
    Y=np.array(labels)
    Y[Y<=threshold] = 0
    # scaler = MinMaxScaler()
    # X=scaler.fit_transform(X)
    x_train_all, X, y_train_all, Y = train_test_split(X, Y, test_size=0.2,random_state=100)
    return X,Y

In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, partial, Trials, STATUS_OK,STATUS_FAIL
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, zero_one_loss,mean_absolute_error,r2_score
import matplotlib.pyplot as plt
import time
from math import sqrt
import os
from tensorflow import keras
import math
import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
print(tf.__version__)
windowsize=19
def CNN_training_best(X, Y):
    # x_train_all, x_predict, y_train_all, y_predict = train_test_split(X, Y, test_size=0.2, random_state=100)
    # del X, Y
    #argsDic=argsDict_tranform(argsDic)
    model=keras.models.Sequential()
    # model.add(LayerNormalization())
    model.add(keras.layers.Reshape((-1, windowsize, 1),input_shape=(19,)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv2D(filters=30,
                  kernel_size=(20, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  ))
    model.add(keras.layers.Conv2D(filters=30,
                  kernel_size=(8, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  ))
    model.add(keras.layers.Conv2D(filters=40,
                  kernel_size=(6, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  ))
    model.add(keras.layers.Conv2D(filters=50,
                  kernel_size=(5, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  ))
    model.add(keras.layers.Conv2D(filters=50,
                  kernel_size=(5, 1),
                  strides=(1, 1),
                  padding='same',
                  activation='relu',
                  ))
    #model.add(LayerNormalization())
    model.add(keras.layers.Flatten(name='flatten'))
    # model.add(keras.layers.Dropout(argsDic['layer3_dropout']))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1024, activation='relu', name='dense'))
    model.add(keras.layers.BatchNormalization())
    # model.add(keras.layers.Dense(256, activation='relu', name='dense1'))
    model.add(keras.layers.Dense(1, activation='linear', name='output'))
    adam = keras.optimizers.Adam(learning_rate=0.0001,
                                  beta_1=0.99,
                                  beta_2=0.999,
                                  epsilon=1e-08)
                                  # use_locking=False)
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mae'])
    # model.compile(optimizer='sgd', loss='mean_absolute_error', metrics=['mae'])
    print('start training')
   # model.fit(X,Y, epochs=50, batch_size=16, validation_split=0.2)
    model.fit(X,Y, epochs=40, batch_size=64,validation_split=0.2)
    return model

2.8.2


In [ ]:
# ######################################Fridge###############################################################
# #trainfile4="/content/drive/MyDrive/training_data/dishwasher_test_.csv"
# trainfile1="/content/drive/MyDrive/training_data/fridge_house_3_training_.csv"

# trainfile2="/content/drive/MyDrive/ukdale_training/fridge_house_2_training_.csv"
# # trainfile3="/content/drive/MyDrive/ukdale_training/fridge_house_2_training_.csv"
# trainfile3="/content/drive/MyDrive/refit_training/fridge/fridge_house_12_training_.csv"
# trainfile4="/content/drive/MyDrive/refit_training/fridge/fridge_house_15_training_.csv"
# fileList=[trainfile1,trainfile2,trainfile3,trainfile4]
# stepsizeList=[2,16,16,16]
# data_insts, data_labels, num_insts,on = [], [], [],[]
# threshold=50
# for i in range(len(fileList)):
#     X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
#     Y=Y.reshape(-1,1)
#     data_insts.append(X)
#     data_labels.append(Y)
#     num_insts.append(X.shape[0])
# min_size=min(num_insts)
# x_tem, y_tem=[],[]
# for j in range(len(fileList)):
#   ridx = np.random.choice(num_insts[j], min_size)
#   x_tem.append(data_insts[j][ridx, :])
#   y_tem.append(data_labels[j][ridx, :])
#   num_insts[j]=x_tem[j].shape[0]
# data_insts=x_tem
# data_labels=y_tem 
# print(num_insts)

In [ ]:
# ######################################Fridge###############################################################
trainfile1="/content/drive/MyDrive/ukdale_training/fridge_house_1_training_.csv"
trainfile2="/content/drive/MyDrive/training_data/fridge_house_2_training_.csv"
trainfile3="/content/drive/MyDrive/refit_training/fridge/fridge_house_2_training_.csv"
trainfile4="/content/drive/MyDrive/refit_training/fridge/fridge_house_5_training_.csv"

trainfile5="/content/drive/MyDrive/ukdale_training/fridge_house_2_training_.csv"
trainfile6="/content/drive/MyDrive/training_data/fridge_test_.csv"
trainfile7="/content/drive/MyDrive/refit_training/fridge/fridge_house_9_training_.csv"
trainfile8="/content/drive/MyDrive/refit_training/fridge/fridge_house_12_training_.csv"
trainfile9="/content/drive/MyDrive/refit_training/fridge/fridge_house_15_training_.csv"
trainfile10="/content/drive/MyDrive/training_data/fridge_house_3_training_.csv"



file=[trainfile1,trainfile2,trainfile3,trainfile4,trainfile5,trainfile6,trainfile7,trainfile8,trainfile9,trainfile10]
stepsizeList=[16,2,16,16,16,2,16,16,16,2]
list_x=[]
list_y=[]
for k in [2,4,6,8,10]:
    data_insts, data_labels, num_insts,on = [], [], [],[]
    threshold=50
    fileList=[]
    for a in range(k):
        fileList.append(file[a])
    for i in range(len(fileList)):
        X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
        Y=Y.reshape(-1,1)
        data_insts.append(X)
        data_labels.append(Y)
        num_insts.append(X.shape[0])

    min_size=min(num_insts)
    x_tem, y_tem=[],[]
    for j in range(len(fileList)):
      ridx = np.random.choice(num_insts[j], min_size)
      x_tem.append(data_insts[j][ridx, :])
      y_tem.append(data_labels[j][ridx, :])
      num_insts[j]=x_tem[j].shape[0]
    data_insts=x_tem
    data_labels=y_tem 
    list_x.append(data_insts)
    list_y.append(data_labels)
    

In [ ]:

expnum=5

for num in range(len(list_x)):
    for n in range(expnum):
        print("start group"+str(num)+",This is the experiment num:", n )
        num_data_sets=len(list_x[num])
        data_insts=list_x[num]
        data_labels=list_y[num]
        num_domains=num_data_sets-1
        for i in [0]:
                print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
                # Build source instances.
                source_insts = []
                source_labels = []
                for j in range(num_data_sets):
                    if j != i:
                        source_insts.append(data_insts[j])
                        source_labels.append(data_labels[j])
                # Build target instances.
                target_idx = i
                target_insts = data_insts[i]
                target_labels = data_labels[i]
                target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
                
                X=np.concatenate(source_insts, axis=0)
                Y=np.concatenate(source_labels, axis=0)
                
                x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
                model=CNN_training_best(X,Y)
                prediction=model.predict(X_t)
                stopT=get_mae(y_t, prediction)
                print(stopT)
                print(get_sae(y_t, prediction))
                print(get_nde(y_t, prediction))



start group0,This is the experiment num: 0

---------------------------------------------- domain num 0 is running----------------------------------
start training
Epoch 1/40
188/188 [==============================] - 8s 36ms/step - loss: 10483.7441 - mae: 80.8984 - val_loss: 13565.4512 - val_mae: 77.0028
Epoch 2/40
188/188 [==============================] - 7s 35ms/step - loss: 8603.5898 - mae: 83.1364 - val_loss: 11789.5156 - val_mae: 81.3435
Epoch 3/40
188/188 [==============================] - 6s 34ms/step - loss: 8258.1416 - mae: 82.5731 - val_loss: 9993.5566 - val_mae: 82.6491
Epoch 4/40
188/188 [==============================] - 6s 34ms/step - loss: 7987.5474 - mae: 81.5373 - val_loss: 7376.9043 - val_mae: 74.6493
Epoch 5/40
188/188 [==============================] - 6s 34ms/step - loss: 7719.8721 - mae: 80.0453 - val_loss: 7078.5488 - val_mae: 74.7593
Epoch 6/40
188/188 [==============================] - 8s 42ms/step - loss: 7424.8491 - mae: 78.1629 - val_loss: 7078.6865 - val_

In [7]:
# ######################################Fridge###############################################################
trainfile1="/content/drive/MyDrive/ukdale_training/washingmachine_house_1_training_.csv"
trainfile2="/content/drive/MyDrive/training_data/washingmachine_house_2_training_.csv"
trainfile3="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_9_training_.csv"
trainfile4="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_15_training_.csv"

trainfile5="/content/drive/MyDrive/ukdale_training/washingmachine_house_2_training_.csv"
trainfile6="/content/drive/MyDrive/training_data/washingmachine_test_.csv"
trainfile7="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_16_training_.csv"
trainfile8="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_17_training_.csv"
trainfile9="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_18_training_.csv"
trainfile10="/content/drive/MyDrive/training_data/washingmachine_house_3_training_.csv"



file=[trainfile1,trainfile2,trainfile3,trainfile4,trainfile5,trainfile6,trainfile7,trainfile8,trainfile9,trainfile10]
stepsizeList=[16,2,16,16,16,2,16,16,16,2]
list_x=[]
list_y=[]
for k in [2,4,6,8,10]:
    data_insts, data_labels, num_insts,on = [], [], [],[]
    threshold=20
    fileList=[]
    for a in range(k):
        fileList.append(file[a])
    for i in range(len(fileList)):
        X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
        Y=Y.reshape(-1,1)
        data_insts.append(X)
        data_labels.append(Y)
        num_insts.append(X.shape[0])

    min_size=min(num_insts)
    x_tem, y_tem=[],[]
    for j in range(len(fileList)):
      ridx = np.random.choice(num_insts[j], min_size)
      x_tem.append(data_insts[j][ridx, :])
      y_tem.append(data_labels[j][ridx, :])
      num_insts[j]=x_tem[j].shape[0]
    data_insts=x_tem
    data_labels=y_tem 
    list_x.append(data_insts)
    list_y.append(data_labels)
    

In [8]:

expnum=5

for num in range(len(list_x)):
    for n in range(expnum):
        print("start group"+str(num)+",This is the experiment num:", n )
        num_data_sets=len(list_x[num])
        data_insts=list_x[num]
        data_labels=list_y[num]
        num_domains=num_data_sets-1
        for i in [0]:
                print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
                # Build source instances.
                source_insts = []
                source_labels = []
                for j in range(num_data_sets):
                    if j != i:
                        source_insts.append(data_insts[j])
                        source_labels.append(data_labels[j])
                # Build target instances.
                target_idx = i
                target_insts = data_insts[i]
                target_labels = data_labels[i]
                target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
                
                X=np.concatenate(source_insts, axis=0)
                Y=np.concatenate(source_labels, axis=0)
                
                x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
                model=CNN_training_best(X,Y)
                prediction=model.predict(X_t)
                stopT=get_mae(y_t, prediction)
                print(stopT)
                print(get_sae(y_t, prediction))
                print(get_nde(y_t, prediction))



start group0,This is the experiment num: 0

---------------------------------------------- domain num 0 is running----------------------------------
start training
Epoch 1/40
188/188 [==============================] - 10s 43ms/step - loss: 6.3972e-04 - mae: 0.0120 - val_loss: 3.5926e-06 - val_mae: 0.0019
Epoch 2/40
188/188 [==============================] - 8s 42ms/step - loss: 2.5549e-05 - mae: 0.0027 - val_loss: 1.3275e-07 - val_mae: 2.9713e-04
Epoch 3/40
188/188 [==============================] - 8s 42ms/step - loss: 6.2905e-06 - mae: 0.0013 - val_loss: 2.8836e-06 - val_mae: 0.0016
Epoch 4/40
188/188 [==============================] - 8s 42ms/step - loss: 2.8822e-06 - mae: 9.1068e-04 - val_loss: 7.3769e-05 - val_mae: 0.0085
Epoch 5/40
188/188 [==============================] - 8s 42ms/step - loss: 1.7359e-06 - mae: 6.6063e-04 - val_loss: 8.2584e-05 - val_mae: 0.0089
Epoch 6/40
188/188 [==============================] - 8s 41ms/step - loss: 1.1189e-06 - mae: 5.6189e-04 - val_loss: 1.

In [ ]:
trainfile1="/content/drive/MyDrive/training_data/fridge_test_.csv"
trainfile2="/content/drive/MyDrive/training_data/fridge_house_2_training_.csv"
trainfile3="/content/drive/MyDrive/ukdale_training/fridge_house_1_training_.csv"
trainfile4="/content/drive/MyDrive/ukdale_training/fridge_house_2_training_.csv"
trainfile5="/content/drive/MyDrive/refit_training/fridge/fridge_house_12_training_.csv"
trainfile6="/content/drive/MyDrive/refit_training/fridge/fridge_house_15_training_.csv"
fileList=[trainfile1,trainfile2,trainfile3,trainfile4,trainfile5,trainfile6]
stepsizeList=[2,2,16,16,16,16]
data_insts, data_labels, num_insts,on = [], [], [],[]
threshold=50
for i in range(len(fileList)):
    X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
    Y=Y.reshape(-1,1)
    data_insts.append(X)
    data_labels.append(Y)
    num_insts.append(X.shape[0])
min_size=min(num_insts)
x_tem, y_tem=[],[]
for j in range(len(fileList)):
  ridx = np.random.choice(num_insts[j], min_size)
  x_tem.append(data_insts[j][ridx, :])
  y_tem.append(data_labels[j][ridx, :])
  num_insts[j]=x_tem[j].shape[0]
data_insts=x_tem
data_labels=y_tem 
print(num_insts)

[15017, 15017, 15017, 15017, 15017, 15017]


In [ ]:
trainfile1="/content/drive/MyDrive/refit_training/fridge/fridge_house_2_training_.csv"
trainfile2="/content/drive/MyDrive/refit_training/fridge/fridge_house_5_training_.csv"
trainfile3="/content/drive/MyDrive/refit_training/fridge/fridge_house_9_training_.csv"
trainfile4="/content/drive/MyDrive/refit_training/fridge/fridge_house_12_training_.csv"
trainfile5="/content/drive/MyDrive/refit_training/fridge/fridge_house_15_training_.csv"
fileList=[trainfile1,trainfile2,trainfile3,trainfile4,trainfile5]
stepsizeList=[16,16,16,16,16]
data_insts, data_labels, num_insts,on = [], [], [],[]
threshold=50
for i in range(len(fileList)):
    X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
    Y=Y.reshape(-1,1)
    data_insts.append(X)
    data_labels.append(Y)
    num_insts.append(X.shape[0])
min_size=min(num_insts)
x_tem, y_tem=[],[]
for j in range(len(fileList)):
  ridx = np.random.choice(num_insts[j], min_size)
  x_tem.append(data_insts[j][ridx, :])
  y_tem.append(data_labels[j][ridx, :])
  num_insts[j]=x_tem[j].shape[0]
data_insts=x_tem
data_labels=y_tem 
print(num_insts)

[71669, 71669, 71669, 71669, 71669]


In [ ]:
for i in [1]:
        print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
        # Build source instances.
        source_insts = []
        source_labels = []
        for j in range(len(fileList)):
            if j != i:
                source_insts.append(data_insts[j])
                source_labels.append(data_labels[j])
        # Build target instances.
        target_idx = i
        target_insts = data_insts[i]
        target_labels = data_labels[i]
        target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
        
        X=np.concatenate(source_insts, axis=0)
        Y=np.concatenate(source_labels, axis=0)
        
        x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
        model=CNN_training_best(X,Y)
        prediction=model.predict(X_t)
        stopT=get_mae(y_t, prediction)
        print(stopT)
        print(get_sae(y_t, prediction))
        print(get_nde(y_t, prediction))



---------------------------------------------- domain num 1 is running----------------------------------
start training
Epoch 1/50
3584/3584 [==============================] - 127s 35ms/step - loss: 2765.6609 - mae: 38.7874 - val_loss: 1500.3828 - val_mae: 35.2442
Epoch 2/50
3584/3584 [==============================] - 123s 34ms/step - loss: 2373.1526 - mae: 38.0297 - val_loss: 2090.2600 - val_mae: 41.8835
Epoch 3/50
3584/3584 [==============================] - 119s 33ms/step - loss: 2368.2993 - mae: 37.8935 - val_loss: 2010.1548 - val_mae: 41.0903
Epoch 4/50
3584/3584 [==============================] - 121s 34ms/step - loss: 2359.9233 - mae: 37.7106 - val_loss: 1812.5397 - val_mae: 39.1214
Epoch 5/50
3584/3584 [==============================] - 119s 33ms/step - loss: 2354.6228 - mae: 37.5632 - val_loss: 2109.7861 - val_mae: 41.9465
Epoch 6/50
3584/3584 [==============================] - 118s 33ms/step - loss: 2353.4556 - mae: 37.5014 - val_loss: 1378.4595 - val_mae: 31.6540
Epoch 7/5

In [ ]:
for i in [0,2,5]:
        print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
        # Build source instances.
        source_insts = []
        source_labels = []
        for j in range(len(fileList)):
            if j != i:
                source_insts.append(data_insts[j])
                source_labels.append(data_labels[j])
        # Build target instances.
        target_idx = i
        target_insts = data_insts[i]
        target_labels = data_labels[i]
        target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
        
        X=np.concatenate(source_insts, axis=0)
        Y=np.concatenate(source_labels, axis=0)
        
        x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
        model=CNN_training_best(X,Y)
        prediction=model.predict(X_t)
        stopT=get_mae(y_t, prediction)
        print(stopT)
        print(get_sae(y_t, prediction))
        print(get_nde(y_t, prediction))


---------------------------------------------- domain num 0 is running----------------------------------
start training
Epoch 1/50
939/939 [==============================] - 62s 64ms/step - loss: 4087.4934 - mae: 42.5080 - val_loss: 1762.5642 - val_mae: 33.8645
Epoch 2/50
939/939 [==============================] - 52s 55ms/step - loss: 3389.8281 - mae: 41.1931 - val_loss: 1835.3457 - val_mae: 34.5247
Epoch 3/50
939/939 [==============================] - 52s 55ms/step - loss: 3353.7578 - mae: 40.9616 - val_loss: 1717.6984 - val_mae: 31.4909
Epoch 4/50
939/939 [==============================] - 51s 54ms/step - loss: 3332.5139 - mae: 40.6672 - val_loss: 1975.4424 - val_mae: 36.4030
Epoch 5/50
939/939 [==============================] - 53s 56ms/step - loss: 3322.2432 - mae: 40.5385 - val_loss: 1910.5995 - val_mae: 35.5875
Epoch 6/50
939/939 [==============================] - 54s 58ms/step - loss: 3303.5430 - mae: 40.2624 - val_loss: 1808.4558 - val_mae: 33.8218
Epoch 7/50
939/939 [=======

In [ ]:
######################################washingmachine###############################################################
# #trainfile4="/content/drive/MyDrive/training_data/dishwasher_test_.csv"
# # trainfile1="/content/drive/MyDrive/training_data/washingmachine_house_2_training_.csv"
# trainfile1="/content/drive/MyDrive/training_data/washingmachine_test_.csv"

# trainfile2="/content/drive/MyDrive/ukdale_training/washingmachine_house_1_training_.csv"
# # trainfile3="/content/drive/MyDrive/ukdale_training/fridge_house_2_training_.csv"
# trainfile3="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_9_training_.csv"
# trainfile4="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_16_training_.csv"

# fileList=[trainfile1,trainfile2,trainfile3,trainfile4]
# stepsizeList=[2,16,16,16]
trainfile1="/content/drive/MyDrive/training_data/washingmachine_test_.csv"
trainfile2="/content/drive/MyDrive/training_data/washingmachine_house_2_training_.csv"
trainfile3="/content/drive/MyDrive/ukdale_training/washingmachine_house_1_training_.csv"
trainfile4="/content/drive/MyDrive/ukdale_training/washingmachine_house_2_training_.csv"
trainfile5="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_9_training_.csv"
trainfile6="/content/drive/MyDrive/refit_training/washingmachine/washingmachine_house_16_training_.csv"
fileList=[trainfile1,trainfile2,trainfile3,trainfile4,trainfile5,trainfile6]
stepsizeList=[2,2,16,16,16,16]
data_insts, data_labels, num_insts,on = [], [], [],[]
threshold=20
for i in range(len(fileList)):
    X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
    Y=Y.reshape(-1,1)
    data_insts.append(X)
    data_labels.append(Y)
    num_insts.append(X.shape[0])
min_size=min(num_insts)
x_tem, y_tem=[],[]
for j in range(len(fileList)):
  ridx = np.random.choice(num_insts[j], min_size)
  x_tem.append(data_insts[j][ridx, :])
  y_tem.append(data_labels[j][ridx, :])
  num_insts[j]=x_tem[j].shape[0]
data_insts=x_tem
data_labels=y_tem 
print(num_insts)

[15017, 15017, 15017, 15017, 15017, 15017]


In [ ]:
for i in [0,2,5]:
        print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
        # Build source instances.
        source_insts = []
        source_labels = []
        for j in range(len(fileList)):
            if j != i:
                source_insts.append(data_insts[j])
                source_labels.append(data_labels[j])
        # Build target instances.
        target_idx = i
        target_insts = data_insts[i]
        target_labels = data_labels[i]
        target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
        
        X=np.concatenate(source_insts, axis=0)
        Y=np.concatenate(source_labels, axis=0)
        
        x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
        model=CNN_training_best(X,Y)
        prediction=model.predict(X_t)
        stopT=get_mae(y_t, prediction)
        print(stopT)
        print(get_sae(y_t, prediction))
        print(get_nde(y_t, prediction))


---------------------------------------------- domain num 0 is running----------------------------------
start training
Epoch 1/50
939/939 [==============================] - 48s 49ms/step - loss: 20229.8105 - mae: 26.0237 - val_loss: 30870.8027 - val_mae: 28.8795
Epoch 2/50
939/939 [==============================] - 47s 50ms/step - loss: 19435.1602 - mae: 24.4780 - val_loss: 29279.5977 - val_mae: 39.1957
Epoch 3/50
939/939 [==============================] - 46s 49ms/step - loss: 19111.0527 - mae: 23.5206 - val_loss: 28688.9707 - val_mae: 38.6389
Epoch 4/50
939/939 [==============================] - 47s 50ms/step - loss: 19090.6816 - mae: 23.6940 - val_loss: 29183.6582 - val_mae: 30.5932
Epoch 5/50
939/939 [==============================] - 45s 48ms/step - loss: 19213.7637 - mae: 24.1038 - val_loss: 28560.5879 - val_mae: 34.7031
Epoch 6/50
939/939 [==============================] - 47s 50ms/step - loss: 19160.4023 - mae: 24.7287 - val_loss: 29185.8535 - val_mae: 31.4334
Epoch 7/50
939/

In [ ]:
######################################dishwasher###############################################################
# #trainfile4="/content/drive/MyDrive/training_data/dishwasher_test_.csv"
# trainfile1="/content/drive/MyDrive/training_data/dishwasher_house_3_training_.csv"

# trainfile2="/content/drive/MyDrive/ukdale_training/dishwasher_house_2_training_.csv"
# # trainfile3="/content/drive/MyDrive/ukdale_training/fridge_house_2_training_.csv"
# trainfile3="/content/drive/MyDrive/refit_training/dishwasher/dishwasher_house_13_training_.csv"
# trainfile4="/content/drive/MyDrive/refit_training/dishwasher/dishwasher_house_18_training_.csv"

# fileList=[trainfile1,trainfile2,trainfile3,trainfile4]
# stepsizeList=[2,16,16,16]
trainfile1="/content/drive/MyDrive/training_data/dishwasher_test_.csv"
trainfile2="/content/drive/MyDrive/training_data/dishwasher_house_2_training_.csv"
trainfile3="/content/drive/MyDrive/ukdale_training/dishwasher_house_1_training_.csv"
trainfile4="/content/drive/MyDrive/ukdale_training/dishwasher_house_2_training_.csv"
trainfile5="/content/drive/MyDrive/refit_training/dishwasher/dishwasher_house_13_training_.csv"
trainfile6="/content/drive/MyDrive/refit_training/dishwasher/dishwasher_house_18_training_.csv"
fileList=[trainfile1,trainfile2,trainfile3,trainfile4,trainfile5,trainfile6]
stepsizeList=[2,2,16,16,16,16]
data_insts, data_labels, num_insts,on = [], [], [],[]
threshold=10
for i in range(len(fileList)):
    X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
    Y=Y.reshape(-1,1)
    data_insts.append(X)
    data_labels.append(Y)
    num_insts.append(X.shape[0])
min_size=min(num_insts)
x_tem, y_tem=[],[]
for j in range(len(fileList)):
  ridx = np.random.choice(num_insts[j], min_size)
  x_tem.append(data_insts[j][ridx, :])
  y_tem.append(data_labels[j][ridx, :])
  num_insts[j]=x_tem[j].shape[0]
data_insts=x_tem
data_labels=y_tem 
print(num_insts)

[15017, 15017, 15017, 15017, 15017, 15017]


In [ ]:
for i in [0,2,4]:
        print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
        # Build source instances.
        source_insts = []
        source_labels = []
        for j in range(len(fileList)):
            if j != i:
                source_insts.append(data_insts[j])
                source_labels.append(data_labels[j])
        # Build target instances.
        target_idx = i
        target_insts = data_insts[i]
        target_labels = data_labels[i]
        target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
        
        X=np.concatenate(source_insts, axis=0)
        Y=np.concatenate(source_labels, axis=0)
        
        x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
        model=CNN_training_best(X,Y)
        prediction=model.predict(X_t)
        stopT=get_mae(y_t, prediction)
        print(stopT)
        print(get_sae(y_t, prediction))
        print(get_nde(y_t, prediction))


---------------------------------------------- domain num 0 is running----------------------------------
start training
Epoch 1/50
939/939 [==============================] - 58s 60ms/step - loss: 40377.1211 - mae: 42.6279 - val_loss: 46188.0977 - val_mae: 51.3632
Epoch 2/50
939/939 [==============================] - 54s 57ms/step - loss: 36168.8477 - mae: 43.9452 - val_loss: 41587.2812 - val_mae: 29.3971
Epoch 3/50
939/939 [==============================] - 55s 59ms/step - loss: 34711.7344 - mae: 43.1153 - val_loss: 38463.7812 - val_mae: 36.6541
Epoch 4/50
939/939 [==============================] - 53s 57ms/step - loss: 33910.8086 - mae: 42.5194 - val_loss: 39172.9141 - val_mae: 46.0548
Epoch 5/50
939/939 [==============================] - 55s 59ms/step - loss: 33853.9492 - mae: 43.2822 - val_loss: 38328.4922 - val_mae: 41.2173
Epoch 6/50
939/939 [==============================] - 52s 56ms/step - loss: 33381.2930 - mae: 43.5909 - val_loss: 39133.0781 - val_mae: 32.6649
Epoch 7/50
939/

In [ ]:
# ######################################microwave###############################################################
#trainfile4="/content/drive/MyDrive/training_data/dishwasher_test_.csv"
# trainfile1="/content/drive/MyDrive/training_data/microwave_house_3_training_.csv"

# trainfile2="/content/drive/MyDrive/ukdale_training/microwave_house_2_training_.csv"
# # trainfile3="/content/drive/MyDrive/ukdale_training/fridge_house_2_training_.csv"
# trainfile3="/content/drive/MyDrive/refit_training/microwave/microwave_house_10_training_.csv"
# trainfile4="/content/drive/MyDrive/refit_training/microwave/microwave_house_17_training_.csv"

# fileList=[trainfile1,trainfile2,trainfile3,trainfile4]
# stepsizeList=[2,16,16,16]

trainfile1="/content/drive/MyDrive/training_data/microwave_test_.csv"
trainfile2="/content/drive/MyDrive/training_data/microwave_house_2_training_.csv"
trainfile3="/content/drive/MyDrive/ukdale_training/microwave_house_1_training_.csv"
trainfile4="/content/drive/MyDrive/ukdale_training/microwave_house_2_training_.csv"
trainfile5="/content/drive/MyDrive/refit_training/microwave/microwave_house_10_training_.csv"
trainfile6="/content/drive/MyDrive/refit_training/microwave/microwave_house_17_training_.csv"
fileList=[trainfile1,trainfile2,trainfile3,trainfile4,trainfile5,trainfile6]
stepsizeList=[2,2,16,16,16,16]
data_insts, data_labels, num_insts,on = [], [], [],[]
threshold=200
for i in range(len(fileList)):
    X,Y=dataProvider(fileList[i], 19, stepsizeList[i], threshold)
    X_on=X[Y>0]
    Y_on=Y[Y>0]
    X_off=X[Y==0]
    Y_off=Y[Y==0]
    ridx = np.random.choice(X_off.shape[0], int(len(Y_on)*50))
    X_off = X_off[ridx, :]
    Y_on=Y_on.reshape(-1,1)
    Y_off=Y_off.reshape(-1,1)
    Y_off = Y_off[ridx, :]
    X_pre=np.concatenate([X_on,X_off], axis=0)
    Y_pre=np.concatenate([Y_on,Y_off], axis=0)
    on.append(Y_on)
    data_insts.append(X_pre)
    data_labels.append(Y_pre)
    num_insts.append(X_pre.shape[0])
    # r_order = np.arange(num_insts[i])
    # np.random.shuffle(r_order)
    # data_insts[i] = data_insts[i][r_order, :]
    # data_labels[i] = data_labels[i][r_order,:]
print(num_insts)
min_size=min(num_insts)
x_tem, y_tem=[],[]
for j in range(len(fileList)):
  ridx = np.random.choice(num_insts[j], min_size)
  x_tem.append(data_insts[j][ridx, :])
  y_tem.append(data_labels[j][ridx, :])
  num_insts[j]=x_tem[j].shape[0]
data_insts=x_tem
data_labels=y_tem 
print(num_insts)

[13311, 2907, 51816, 4335, 11475, 10302]
[2907, 2907, 2907, 2907, 2907, 2907]


In [ ]:
fileList

NameError: ignored

In [ ]:
for i in [0,2,4]:
        print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
        # Build source instances.
        source_insts = []
        source_labels = []
        for j in range(len(fileList)):
            if j != i:
                source_insts.append(data_insts[j])
                source_labels.append(data_labels[j])
        # Build target instances.
        target_idx = i
        target_insts = data_insts[i]
        target_labels = data_labels[i]
        target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
        
        X=np.concatenate(source_insts, axis=0)
        Y=np.concatenate(source_labels, axis=0)
        
        x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
        model=CNN_training_best(X,Y)
        prediction=model.predict(X_t)
        stopT=get_mae(y_t, prediction)
        print(stopT)
        print(get_sae(y_t, prediction))
        print(get_nde(y_t, prediction))


---------------------------------------------- domain num 0 is running----------------------------------
start training
Epoch 1/50
271/271 [==============================] - 15s 43ms/step - loss: 36427.3281 - mae: 28.5333 - val_loss: 28900.7363 - val_mae: 23.8294
Epoch 2/50
271/271 [==============================] - 10s 38ms/step - loss: 33981.1445 - mae: 41.4449 - val_loss: 26373.3496 - val_mae: 48.0415
Epoch 3/50
271/271 [==============================] - 10s 39ms/step - loss: 32483.1992 - mae: 46.5032 - val_loss: 25659.5020 - val_mae: 55.1275
Epoch 4/50
271/271 [==============================] - 10s 38ms/step - loss: 31174.6406 - mae: 45.4867 - val_loss: 25180.7324 - val_mae: 49.8655
Epoch 5/50
271/271 [==============================] - 10s 38ms/step - loss: 29093.2500 - mae: 40.9751 - val_loss: 25293.7539 - val_mae: 56.6046
Epoch 6/50
271/271 [==============================] - 10s 38ms/step - loss: 27862.7598 - mae: 40.1446 - val_loss: 25343.9434 - val_mae: 52.3048
Epoch 7/50
271/

In [ ]:
for i in [5]:
        print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
        # Build source instances.
        source_insts = []
        source_labels = []
        for j in range(len(fileList)):
            if j != i:
                source_insts.append(data_insts[j])
                source_labels.append(data_labels[j])
        # Build target instances.
        target_idx = i
        target_insts = data_insts[i]
        target_labels = data_labels[i]
        target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
        
        X=np.concatenate(source_insts, axis=0)
        Y=np.concatenate(source_labels, axis=0)
        
        x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
        model=CNN_training_best(X,Y)
        prediction=model.predict(X_t)
        stopT=get_mae(y_t, prediction)
        print(stopT)
        print(get_sae(y_t, prediction))
        print(get_nde(y_t, prediction))


---------------------------------------------- domain num 5 is running----------------------------------
start training
Epoch 1/50
271/271 [==============================] - 14s 45ms/step - loss: 42156.0547 - mae: 32.3752 - val_loss: 18957.1836 - val_mae: 23.7764
Epoch 2/50
271/271 [==============================] - 10s 38ms/step - loss: 37926.8828 - mae: 42.4810 - val_loss: 21506.8613 - val_mae: 69.8233
Epoch 3/50
271/271 [==============================] - 10s 39ms/step - loss: 35305.9922 - mae: 45.1648 - val_loss: 25047.3281 - val_mae: 79.7593
Epoch 4/50
271/271 [==============================] - 11s 39ms/step - loss: 33525.7891 - mae: 44.6917 - val_loss: 28194.4629 - val_mae: 76.2965
Epoch 5/50
271/271 [==============================] - 10s 39ms/step - loss: 32428.7812 - mae: 44.5551 - val_loss: 24148.4531 - val_mae: 78.4546
Epoch 6/50
271/271 [==============================] - 11s 39ms/step - loss: 30504.6641 - mae: 41.4045 - val_loss: 26281.7539 - val_mae: 77.1783
Epoch 7/50
271/

In [ ]:
for i in [1,3]:
        print('\n---------------------------------------------- domain num %i is running----------------------------------'%(i))
        # Build source instances.
        source_insts = []
        source_labels = []
        for j in range(len(fileList)):
            if j != i:
                source_insts.append(data_insts[j])
                source_labels.append(data_labels[j])
        # Build target instances.
        target_idx = i
        target_insts = data_insts[i]
        target_labels = data_labels[i]
        target_insts0, X_t, target_labels0, y_t =train_test_split(target_insts, target_labels, test_size=0.2,random_state=100)
        
        X=np.concatenate(source_insts, axis=0)
        Y=np.concatenate(source_labels, axis=0)
        
        x_train_all, x_test, y_train_all, y_test =train_test_split(X, Y, test_size=0.2,random_state=100)
        model=CNN_training_best(X,Y)
        prediction=model.predict(X_t)
        stopT=get_mae(y_t, prediction)
        print(stopT)
        print(get_sae(y_t, prediction))
        print(get_nde(y_t, prediction))


---------------------------------------------- domain num 1 is running----------------------------------
start training
Epoch 1/50
271/271 [==============================] - 11s 36ms/step - loss: 35721.8750 - mae: 30.3331 - val_loss: 36448.5195 - val_mae: 29.6640
Epoch 2/50
271/271 [==============================] - 10s 36ms/step - loss: 33448.1055 - mae: 44.8122 - val_loss: 32956.7031 - val_mae: 53.0786
Epoch 3/50
271/271 [==============================] - 9s 32ms/step - loss: 32193.0781 - mae: 47.3426 - val_loss: 31067.2676 - val_mae: 51.3792
Epoch 4/50
271/271 [==============================] - 9s 34ms/step - loss: 31267.5508 - mae: 48.3951 - val_loss: 30838.6660 - val_mae: 42.7676
Epoch 5/50
271/271 [==============================] - 10s 37ms/step - loss: 29910.1094 - mae: 42.9760 - val_loss: 32555.1250 - val_mae: 69.5932
Epoch 6/50
271/271 [==============================] - 9s 34ms/step - loss: 28818.5684 - mae: 43.5400 - val_loss: 32053.5371 - val_mae: 56.5072
Epoch 7/50
271/271